# Zomato_Delivery_Time_Predictions

Importing the libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')

Importing train dataset

In [2]:
train= pd.read_excel('Data_Train.xlsx')
train.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",₹200,₹50,3.5,12,4,30 minutes
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",₹100,₹50,3.5,11,4,30 minutes
2,ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",₹150,₹50,3.6,99,30,65 minutes
3,ID_5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",₹250,₹99,3.7,176,95,30 minutes
4,ID_6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",₹200,₹99,3.2,521,235,65 minutes


Now let's have a look on the dataset

In [3]:
print(train.shape)

(11094, 9)


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11094 entries, 0 to 11093
Data columns (total 9 columns):
Restaurant       11094 non-null object
Location         11094 non-null object
Cuisines         11094 non-null object
Average_Cost     11094 non-null object
Minimum_Order    11094 non-null object
Rating           11094 non-null object
Votes            11094 non-null object
Reviews          11094 non-null object
Delivery_Time    11094 non-null object
dtypes: object(9)
memory usage: 780.2+ KB


##  Data Pre-procesing on Train Data

Removing ₹ sign from Average_Cost & Minimum_Order columns and minutes from Delivery_Time column

In [5]:
train["Average_Cost"]= train["Average_Cost"].str.replace("₹","")
train["Minimum_Order"]= train["Minimum_Order"].str.replace("₹","")
train["Delivery_Time"]= train["Delivery_Time"].str.replace("minutes","")

Here we pick out cities from the locations and have made a another column of City 

In [6]:
train['City.Pune'] = train['Location'].apply(lambda x: 'Pune' if 'Pune' in x else None)
train['City.Kolkata'] = train['Location'].apply(lambda x: 'Kolkata' if 'Kolkata' in x else None)
train['City.Mumbai'] = train['Location'].apply(lambda x: 'Mumbai' if 'Mumbai' in x else None)
train['City.Bangalore'] = train['Location'].apply(lambda x: 'Bangalore' if 'Bangalore' in x else None)
train['City.Delhi'] = train['Location'].apply(lambda x: 'Delhi' if 'Delhi' in x else None)
train['City.Hyderabad'] = train['Location'].apply(lambda x: 'Hyderabad' if 'Hyderabad' in x else None)
train['City.Noida'] = train['Location'].apply(lambda x: 'Noida' if 'Noida' in x else None)
train['City.Gurgaon'] = train['Location'].apply(lambda x: 'Gurgaon' if 'Gurgaon' in x else None)
train['City.Majestic'] = train['Location'].apply(lambda x: 'Bangalore' if 'Majestic' in x else None)
train['City.Marathalli'] = train['Location'].apply(lambda x: 'Bangalore' if 'Marathalli' in x else None)
train['City.Electronic'] = train['Location'].apply(lambda x: 'Bangalore' if 'Electronic' in x else None)
train['City.Gurgoan'] = train['Location'].apply(lambda x: 'Gurgaon' if 'Gurgoan' in x else None)
train['City.Whitefield'] = train['Location'].apply(lambda x: 'Bangalore' if 'Whitefield' in x else None)

train['City'] = train['City.Pune'].map(str)+train['City.Kolkata'].map(str)+train['City.Mumbai'].map(str)+train['City.Bangalore'].map(str)+train['City.Delhi'].map(str)+train['City.Hyderabad'].map(str)+train['City.Noida'].map(str)+train['City.Gurgaon'].map(str)+train['City.Majestic'].map(str)+train['City.Marathalli'].map(str)+train['City.Electronic'].map(str)+train['City.Gurgoan'].map(str)+train['City.Whitefield'].map(str)

train['City'] = train['City'].apply(lambda x: x.replace('None',''))

In [7]:
train = train[['Restaurant','City','Cuisines','Average_Cost', 'Minimum_Order', 'Rating', 'Votes', 'Reviews', 'Delivery_Time']]
train.head()

,Restaurant,City,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
0,ID_6321,Pune,"Fast Food, Rolls, Burger, Salad, Wraps",200,50,3.5,12,4,30
1,ID_2882,Bangalore,"Ice Cream, Desserts",100,50,3.5,11,4,30
2,ID_1595,Mumbai,"Italian, Street Food, Fast Food",150,50,3.6,99,30,65
3,ID_5929,Noida,"Mughlai, North Indian, Chinese",250,99,3.7,176,95,30
4,ID_6123,Bangalore,"Cafe, Beverages",200,99,3.2,521,235,65


Here we count the Cuisines in each restaurant

In [8]:
def countoc(s):
    a=s.split(',')
    return len(a)

train['Count Cuisine'] = train['Cuisines'].apply(lambda x:countoc(x))
train.head()

,Restaurant,City,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time,Count Cuisine
0,ID_6321,Pune,"Fast Food, Rolls, Burger, Salad, Wraps",200,50,3.5,12,4,30,5
1,ID_2882,Bangalore,"Ice Cream, Desserts",100,50,3.5,11,4,30,2
2,ID_1595,Mumbai,"Italian, Street Food, Fast Food",150,50,3.6,99,30,65,3
3,ID_5929,Noida,"Mughlai, North Indian, Chinese",250,99,3.7,176,95,30,3
4,ID_6123,Bangalore,"Cafe, Beverages",200,99,3.2,521,235,65,2


Converting object to numeric

In [9]:
cols = ['Average_Cost', 'Minimum_Order', 'Rating', 'Votes', 'Reviews', 'Delivery_Time']
train[cols] = train[cols].apply(pd.to_numeric,errors='coerce')

In [10]:
train.isnull().sum()

Restaurant          0
City                0
Cuisines            0
Average_Cost       25
Minimum_Order       0
Rating           1963
Votes            2074
Reviews          2312
Delivery_Time       0
Count Cuisine       0
dtype: int64

In [11]:
train.drop(['Restaurant','Cuisines'], axis=1, inplace=True)

In [12]:
train=pd.get_dummies(data=train, drop_first=True)
train.head()

,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time,Count Cuisine,City_Delhi,City_Gurgaon,City_Hyderabad,City_Kolkata,City_Mumbai,City_Noida,City_Pune
0,200.0,50,3.5,12.0,4.0,30,5,0,0,0,0,0,0,1
1,100.0,50,3.5,11.0,4.0,30,2,0,0,0,0,0,0,0
2,150.0,50,3.6,99.0,30.0,65,3,0,0,0,0,1,0,0
3,250.0,99,3.7,176.0,95.0,30,3,0,0,0,0,0,1,0
4,200.0,99,3.2,521.0,235.0,65,2,0,0,0,0,0,0,0


Using KNN Imputer to deal with missing values

In [13]:
imputer = KNNImputer(n_neighbors=3, weights="uniform")
train=imputer.fit_transform(train)
train=pd.DataFrame(train)

In [14]:
train.columns=['Average_Cost', 'Minimum_Order', 'Rating', 'Votes', 'Reviews',
       'Delivery_Time', 'Count Cuisine', 'City_Delhi', 'City_Gurgaon',
       'City_Hyderabad', 'City_Kolkata', 'City_Mumbai', 'City_Noida',
       'City_Pune']
train.head()

,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time,Count Cuisine,City_Delhi,City_Gurgaon,City_Hyderabad,City_Kolkata,City_Mumbai,City_Noida,City_Pune
0,200.0,50.0,3.5,12.0,4.0,30.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,100.0,50.0,3.5,11.0,4.0,30.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,150.0,50.0,3.6,99.0,30.0,65.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,250.0,99.0,3.7,176.0,95.0,30.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,200.0,99.0,3.2,521.0,235.0,65.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Preparing our Dataset for ML

In [15]:
X=train.drop(['Delivery_Time'],axis=1).values
y=train['Delivery_Time'].values

In [16]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

# KNN 

In [17]:
model_knn = KNeighborsClassifier(n_neighbors=1)
model_knn.fit(X_train, y_train)

y_test_pred_knn = model_knn.predict(X_test)
score = model_knn.score(X_test, y_test)

acc_knn="{0:.2f}%".format(100*accuracy_score(y_test_pred_knn, y_test))
print("Accuracy {0:.2f}%".format(100*accuracy_score(y_test_pred_knn, y_test)))
print(confusion_matrix(y_test, y_test_pred_knn))
print(classification_report(y_test, y_test_pred_knn))

Accuracy 74.50%
[[   1    0    0    0    0    0    0]
 [   0    0    4    1    0    0    0]
 [   0    5 1882  229   75    0    2]
 [   1    0  278  469   76    0    2]
 [   0    2   89   68  120    0    0]
 [   0    0    2    1    0    2    0]
 [   0    0    9    5    0    0    6]]
              precision    recall  f1-score   support

        10.0       0.50      1.00      0.67         1
        20.0       0.00      0.00      0.00         5
        30.0       0.83      0.86      0.84      2193
        45.0       0.61      0.57      0.59       826
        65.0       0.44      0.43      0.44       279
        80.0       1.00      0.40      0.57         5
       120.0       0.60      0.30      0.40        20

    accuracy                           0.74      3329
   macro avg       0.57      0.51      0.50      3329
weighted avg       0.74      0.74      0.74      3329



# XGBoost 

In [18]:
model_xgb = XGBClassifier(learning_rate= 1,random_state=0)
model_xgb.fit(X_train, y_train)

y_test_pred_xgb = model_xgb.predict(X_test)
score = model_xgb.score(X_test, y_test)

acc_xgb="{0:.2f}%".format(100*accuracy_score(y_test_pred_xgb, y_test))
print("Accuracy {0:.2f}%".format(100*accuracy_score(y_test_pred_xgb, y_test)))
print(confusion_matrix(y_test, y_test_pred_xgb))
print(classification_report(y_test, y_test_pred_xgb))

Accuracy 74.89%
[[   1    0    0    0    0    0    0]
 [   0    0    4    1    0    0    0]
 [   0    0 2015  157   20    0    1]
 [   0    1  372  415   37    0    1]
 [   0    1  118  105   55    0    0]
 [   0    0    3    0    0    2    0]
 [   0    0   10    3    2    0    5]]
              precision    recall  f1-score   support

        10.0       1.00      1.00      1.00         1
        20.0       0.00      0.00      0.00         5
        30.0       0.80      0.92      0.85      2193
        45.0       0.61      0.50      0.55       826
        65.0       0.48      0.20      0.28       279
        80.0       1.00      0.40      0.57         5
       120.0       0.71      0.25      0.37        20

    accuracy                           0.75      3329
   macro avg       0.66      0.47      0.52      3329
weighted avg       0.72      0.75      0.73      3329



In [19]:
parameters=[{'learning_rate':[0.6,0.8,0.99,1,2],'random_state':[0,1,2,3]}]

grid_search= GridSearchCV(estimator= model_xgb,param_grid=parameters,scoring='accuracy',cv=5,n_jobs= -1)

grid_search=grid_search.fit(X_train, y_train)
best_accuracy_xgb=grid_search.best_score_
best_parameters_xgb=grid_search.best_params_

In [20]:
print(best_accuracy_xgb)
print(best_parameters_xgb)

0.7419188667095943
{'learning_rate': 0.6, 'random_state': 0}


# Random Forest 

In [21]:
model_rf = RandomForestClassifier(criterion='entropy', n_estimators=40, random_state=2)
model_rf.fit(X_train, y_train)

y_test_pred_rf = model_rf.predict(X_test)
score = model_rf.score(X_test, y_test)

acc_rf="{0:.2f}%".format(100*accuracy_score(y_test_pred_rf, y_test))
print("Accuracy {0:.2f}%".format(100*accuracy_score(y_test_pred_rf, y_test)))
print(confusion_matrix(y_test, y_test_pred_rf))
print(classification_report(y_test, y_test_pred_rf))

Accuracy 78.97%
[[   1    0    0    0    0    0    0]
 [   0    0    4    1    0    0    0]
 [   0    0 2041  138   12    0    2]
 [   0    0  318  481   27    0    0]
 [   0    0   90   90   99    0    0]
 [   0    0    2    1    0    2    0]
 [   0    0    7    8    0    0    5]]
              precision    recall  f1-score   support

        10.0       1.00      1.00      1.00         1
        20.0       0.00      0.00      0.00         5
        30.0       0.83      0.93      0.88      2193
        45.0       0.67      0.58      0.62       826
        65.0       0.72      0.35      0.47       279
        80.0       1.00      0.40      0.57         5
       120.0       0.71      0.25      0.37        20

    accuracy                           0.79      3329
   macro avg       0.70      0.50      0.56      3329
weighted avg       0.78      0.79      0.78      3329



In [22]:
from sklearn.model_selection import GridSearchCV

parameters=[{'n_estimators' : [20,30,40,50], 'criterion' : ['entropy'],'random_state':[0,1,2]},
           {'n_estimators' : [20,30,40,50], 'criterion' : ['gini'],'random_state':[0,1,2]}]

grid_search= GridSearchCV(estimator= model_rf,param_grid=parameters,scoring='accuracy',cv=5,n_jobs= -1)

grid_search=grid_search.fit(X_train,y_train)
best_accuracy_rf=grid_search.best_score_
best_parameters_rf=grid_search.best_params_

In [23]:
print(best_accuracy_rf)
print(best_parameters_rf)

0.7817128139085641
{'criterion': 'gini', 'n_estimators': 40, 'random_state': 1}


# SVC 

In [24]:
model_svc = SVC(kernel = 'rbf',gamma=0.8, random_state=0)
model_svc.fit(X_train, y_train)

y_test_pred_svc = model_svc.predict(X_test)
score = model_svc.score(X_test, y_test)

acc_svc="{0:.2f}%".format(100*accuracy_score(y_test_pred_svc, y_test))
print("Accuracy {0:.2f}%".format(100*accuracy_score(y_test_pred_svc, y_test)))
print(confusion_matrix(y_test, y_test_pred_svc))
print(classification_report(y_test, y_test_pred_svc))


Accuracy 75.73%
[[   0    0    1    0    0    0    0]
 [   0    0    5    0    0    0    0]
 [   0    0 2186    5    2    0    0]
 [   0    0  585  241    0    0    0]
 [   0    0  190    2   87    0    0]
 [   0    0    3    0    0    2    0]
 [   0    0   15    0    0    0    5]]
              precision    recall  f1-score   support

        10.0       0.00      0.00      0.00         1
        20.0       0.00      0.00      0.00         5
        30.0       0.73      1.00      0.84      2193
        45.0       0.97      0.29      0.45       826
        65.0       0.98      0.31      0.47       279
        80.0       1.00      0.40      0.57         5
       120.0       1.00      0.25      0.40        20

    accuracy                           0.76      3329
   macro avg       0.67      0.32      0.39      3329
weighted avg       0.81      0.76      0.71      3329



In [25]:
parameters = [{'C': [1, 10, 100], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}]

grid_search = GridSearchCV(estimator = model_svc,param_grid = parameters,scoring = 'accuracy',cv = 5,n_jobs = -1)

grid_search=grid_search.fit(X_train, y_train)
best_accuracy_svc=grid_search.best_score_
best_parameters_svc=grid_search.best_params_



In [26]:
print(best_accuracy_svc)
print(best_parameters_svc)

0.7502897617514488
{'C': 1, 'gamma': 0.8, 'kernel': 'rbf'}


## Data Pre-processing on Test Data

In [27]:
test= pd.read_excel('Data_Test.xlsx')
test_d = test.copy()
test.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews
0,ID_2842,"Mico Layout, Stage 2, BTM Layout,Bangalore","North Indian, Chinese, Assamese",₹350,₹50,4.2,361,225
1,ID_730,"Mico Layout, Stage 2, BTM Layout,Bangalore","Biryani, Kebab",₹100,₹50,NEW,-,-
2,ID_4620,"Sector 1, Noida",Fast Food,₹100,₹50,3.6,36,16
3,ID_5470,"Babarpur, New Delhi, Delhi","Mithai, North Indian, Chinese, Fast Food, Sout...",₹200,₹50,3.6,66,33
4,ID_3249,"Sector 1, Noida","Chinese, Fast Food",₹150,₹50,2.9,38,14


In [28]:
test["Average_Cost"]= test["Average_Cost"].str.replace("₹","")
test["Minimum_Order"]= test["Minimum_Order"].str.replace("₹","")
test["Votes"]= test["Votes"].str.replace("-","")
test["Reviews"]= test["Reviews"].str.replace("-","")
test["Rating"]= test["Rating"].str.replace("-","")
test["Rating"]= test["Rating"].str.replace("NEW","")
test["Rating"]= test["Rating"].str.replace("Opening Soon","")

In [29]:
test['City.Pune'] = test['Location'].apply(lambda x: 'Pune' if 'Pune' in x else None)
test['City.Kolkata'] = test['Location'].apply(lambda x: 'Kolkata' if 'Kolkata' in x else None)
test['City.Mumbai'] = test['Location'].apply(lambda x: 'Mumbai' if 'Mumbai' in x else None)
test['City.Bangalore'] = test['Location'].apply(lambda x: 'Bangalore' if 'Bangalore' in x else None)
test['City.Delhi'] = test['Location'].apply(lambda x: 'Delhi' if 'Delhi' in x else None)
test['City.Hyderabad'] = test['Location'].apply(lambda x: 'Hyderabad' if 'Hyderabad' in x else None)
test['City.Noida'] = test['Location'].apply(lambda x: 'Noida' if 'Noida' in x else None)
test['City.Gurgaon'] = test['Location'].apply(lambda x: 'Gurgaon' if 'Gurgaon' in x else None)
test['City.Majestic'] = test['Location'].apply(lambda x: 'Bangalore' if 'Majestic' in x else None)
test['City.Marathalli'] = test['Location'].apply(lambda x: 'Bangalore' if 'Marathalli' in x else None)
test['City.Electronic'] = test['Location'].apply(lambda x: 'Bangalore' if 'Electronic' in x else None)
test['City.Gurgoan'] = test['Location'].apply(lambda x: 'Gurgaon' if 'Gurgoan' in x else None)
test['City.Whitefield'] = test['Location'].apply(lambda x: 'Bangalore' if 'Whitefield' in x else None)

test['City'] = test['City.Pune'].map(str)+test['City.Kolkata'].map(str)+test['City.Mumbai'].map(str)+test['City.Bangalore'].map(str)+test['City.Delhi'].map(str)+test['City.Hyderabad'].map(str)+test['City.Noida'].map(str)+test['City.Gurgaon'].map(str)+test['City.Majestic'].map(str)+test['City.Marathalli'].map(str)+test['City.Electronic'].map(str)+test['City.Gurgoan'].map(str)+test['City.Whitefield'].map(str)

test['City'] = test['City'].apply(lambda x: x.replace('None',''))

In [30]:
test = test[['Restaurant','City','Cuisines','Average_Cost', 'Minimum_Order', 'Rating', 'Votes', 'Reviews']]
test.head()

,Restaurant,City,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews
0,ID_2842,Bangalore,"North Indian, Chinese, Assamese",350,50,4.2,361,225
1,ID_730,Bangalore,"Biryani, Kebab",100,50,,,
2,ID_4620,Noida,Fast Food,100,50,3.6,36,16
3,ID_5470,Delhi,"Mithai, North Indian, Chinese, Fast Food, Sout...",200,50,3.6,66,33
4,ID_3249,Noida,"Chinese, Fast Food",150,50,2.9,38,14


In [31]:
def countoc(s):
    a=s.split(',')
    return len(a)

test['Count Cuisine'] = test['Cuisines'].apply(lambda x:countoc(x))

cols_t= ['Average_Cost', 'Minimum_Order', 'Rating', 'Votes', 'Reviews']
test[cols_t] = test[cols_t].apply(pd.to_numeric,errors='coerce')

In [32]:
test.head()

,Restaurant,City,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Count Cuisine
0,ID_2842,Bangalore,"North Indian, Chinese, Assamese",350.0,50,4.2,361.0,225.0,3
1,ID_730,Bangalore,"Biryani, Kebab",100.0,50,NaN,NaN,NaN,2
2,ID_4620,Noida,Fast Food,100.0,50,3.6,36.0,16.0,1
3,ID_5470,Delhi,"Mithai, North Indian, Chinese, Fast Food, Sout...",200.0,50,3.6,66.0,33.0,5
4,ID_3249,Noida,"Chinese, Fast Food",150.0,50,2.9,38.0,14.0,2


In [33]:
test.isnull().sum()

Restaurant         0
City               0
Cuisines           0
Average_Cost       6
Minimum_Order      0
Rating           507
Votes            542
Reviews          593
Count Cuisine      0
dtype: int64

In [34]:
test.drop(['Restaurant','Cuisines'], axis=1, inplace=True)

In [35]:
test=pd.get_dummies(data=test, drop_first=True)

In [36]:
imputer = KNNImputer(n_neighbors=3, weights="uniform")
test=imputer.fit_transform(test)
test=pd.DataFrame(test)

In [37]:
test.columns=['Average_Cost', 'Minimum_Order', 'Rating', 'Votes', 'Reviews',
       'Count Cuisine', 'City_Delhi', 'City_Gurgaon', 'City_Hyderabad',
       'City_Kolkata', 'City_Mumbai', 'City_Noida', 'City_Pune']
test.head()

,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Count Cuisine,City_Delhi,City_Gurgaon,City_Hyderabad,City_Kolkata,City_Mumbai,City_Noida,City_Pune
0,350.0,50.0,4.2,361.000000,225.000000,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100.0,50.0,3.4,37.333333,16.333333,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100.0,50.0,3.6,36.000000,16.000000,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,200.0,50.0,3.6,66.000000,33.000000,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,150.0,50.0,2.9,38.000000,14.000000,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [38]:
X1 = test[['Average_Cost', 'Minimum_Order', 'Rating', 'Votes', 'Reviews',
       'Count Cuisine', 'City_Delhi', 'City_Gurgaon', 'City_Hyderabad',
       'City_Kolkata', 'City_Mumbai', 'City_Noida', 'City_Pune']]

#### Running model on test data and generating predictions with Random Forest classifier Model as it is giving the better accuracy precision

In [39]:
y_pred_rf = model_rf.predict(X1)

In [40]:
sub=pd.DataFrame()
sub["Restaurant"]=test_d["Restaurant"]
sub["Delivery_Time"]=np.round(y_pred_rf).astype(int)
sub["Delivery_Time"]=sub["Delivery_Time"].astype(str)+" minutes"
sub.to_excel("output.xlsx",index=False)
sub

,Restaurant,Delivery_Time
0,ID_2842,30 minutes
1,ID_730,30 minutes
2,ID_4620,30 minutes
3,ID_5470,30 minutes
4,ID_3249,30 minutes
...,...,...
2769,ID_6342,30 minutes
2770,ID_8495,30 minutes
2771,ID_7122,45 minutes
2772,ID_2475,30 minutes
